# Config environment

In [ ]:
path = '/content/drive/MyDrive/reaction_balancer'
import sys
sys.path.append(path)
%cd /content/drive/MyDrive/reaction_balancer

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/content/drive/MyDrive/masked_smiles
Fri Feb  3 09:55:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                   

In [ ]:
# %%bash
!pip install torch torchvision
!pip install pandas
!pip install transformers
!pip install wandb

from src.main_functions import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.6 MB/s 
     |████████████████████████████████| 7.6 MB 59.0 MB/s 
     |████████████████████████████████| 182 kB 80.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 34.0 MB/s 
     |████████████████████████████████| 182 kB 74.1 MB/s 
     |████████████████████████████████| 168 kB 76.4 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 168 kB 82.4 MB/s 
     |████████████████████████████████| 166 kB 72.1 MB/s 
     |████████████████████████████████| 166 kB 80.4 MB/s 
     |█████████████████████

# Config wandb

wandb config with HuggingFace

https://docs.wandb.ai/guides/integrations/huggingface

In [ ]:
!wandb login

import wandb
wandb.login()
wandb.init()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chonghuanzhang. Use `wandb login --relogin` to force relogin


# Define dataset and method to learn

Datasets:

(1) learn from uspto: uspto_sentences.txt, uspto_masked_smiles.csv

(2) learn from reaxys + uspto: complete_sentences.txt, complete_masked_smiles.csv

(3) learn from uspto_cleaned: uspto_cleaned_sentence.txt, uspto_cleaned_masked_smiles.csv

This is the cleaned data by removing unmapped reactant into reagents, and passed through the ChemBalancer



Methods:

(1) random mask: conventional way to mask 15% of reaction SMILES dynamically, no matter if that crosses "." and ">>".

(2) default mask: mask the specific defined molecule in reaction SMILES.

In [ ]:
learn_from = 'uspto' # 'complete'

mask_method = 'random' #'default'

# trial number
trial_num = 6

# Choose if the ChemMLM model is LHS ChemMLM or LHS ChemMLM.
mask_side = 'right' # 'left'

In [ ]:
data_file =  'data/{}_sentences.txt'.format(learn_from)
tokenizer_file = "tokenizer/{}_custom_tokenizer".format(learn_from)
pretrain_tokenizer_file = "tokenizer/{}_tokenizer".format(learn_from)
if mask_side == 'right':
    masked_data_file = 'data/{}_masked_token_smiles.csv'.format(learn_from)
else:
    masked_data_file = 'data/{}_masked_token_smiles_lhs.csv'.format(learn_from)

# From saved tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrain_tokenizer_file)

# Tokenize reaction SMILES

In [ ]:
tokenizer = mlm_tokenizer(data_file, tokenizer_file, pretrain_tokenizer_file)

# Train masked lanuage model

In [ ]:
config = RobertaConfig(
    vocab_size=52000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
configuration = RobertaConfig()
print(configuration)

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



具体可以配置哪些参数可以通过默认参数的定义来自行定义

In [ ]:
model = RobertaForMaskedLM(config=config)

print(model.num_parameters())

83504416


TrainingArguments里设置gradient_accumulation_steps = 10 则batch size 可以设置为160*10 = 160 本质上就是16个样本算一次梯度，但是累计计算了10次之后才反向传播 这样得到的效果和一次性算160个样本的梯度并反向传播是一样的

In [ ]:
training_args = TrainingArguments(
    output_dir="model/{}_{}_mlm_#{}".format(learn_from, mask_method, trial_num),
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_device_train_batch_size=16,
    save_steps=20000,
    save_total_limit=None,
    remove_unused_columns=False,
    learning_rate=2e-04,
    gradient_accumulation_steps=1,
    # evaludation
    evaluation_strategy='steps',
    eval_steps=5000,
    eval_accumulation_steps=1,
    # wandb
    report_to="wandb",  # enable logging to W&B
    run_name="{}_{}_mlm_#{}".format(learn_from, mask_method, trial_num)  # name of the W&B run (optional)
)

if mask_method == 'default':
    data_collator = CustomDataCollatorForLanguageModeling(tokenizer)

    dataset_trn, dataset_evl, dataset_tst = get_dataset(
    mask_method, masked_data_file, data_file, tokenizer)

elif mask_method == 'random':
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.5)

    if learn_from == 'complete':
        dataset_trn, dataset_evl = get_complete_data(mask_method, masked_data_file, data_file, tokenizer, mode='train')
        # dataset_trn = pd.read_pickle(os.path.join('./data/complete_sentences/', 'dataset_trn.pickle'))
        # dataset_evl = pd.read_pickle(os.path.join('./data/complete_sentences/', 'dataset_evl.pickle'))
        # dataset_tst = pd.read_pickle(os.path.join('./data/complete_sentences/', 'dataset_tst.pickle'))

    else:
        # get data
        dataset_trn, dataset_evl, dataset_tst = get_dataset(
            mask_method, masked_data_file, data_file, tokenizer)

#DefaultDataCollator(return_tensors='pt')
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    # dataset=dataset,
    train_dataset=dataset_trn,
    eval_dataset=dataset_evl,
)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
import torch
dd = trainer.get_train_dataloader()
ddd = next(iter(dd))
print(ddd)

for k in ddd:
    print(k)
    print(ddd[k].shape)

{'input_ids': tensor([[2863,   58,   34,  ..., 2861, 2861, 2861],
        [ 273, 2863, 2863,  ..., 2861, 2861, 2861],
        [2863, 2863,   46,  ..., 2861, 2861, 2861],
        ...,
        [ 257, 2863, 2863,  ..., 2861, 2861, 2861],
        [  45, 2594, 2863,  ..., 2861, 2861, 2861],
        [ 257,    7,   34,  ..., 2863, 2302, 2863]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  34, -100,   34,  ..., -100, -100, -100],
        [-100,   16,  256,  ..., -100, -100, -100],
        [ 287,  259, -100,  ..., -100, -100, -100],
        ...,
        [-100,  259,   46,  ..., -100, -100, -100],
        [  45,    2,  265,  ..., -100, -100, -100],
        [-100, -100, -100,  ...,   46,   28,   46]])}
input_ids
torch.Size([16, 231])
attention_mask
torch.Size([16, 231])
labels
torch.Siz

In [ ]:
%%time
# trainer.train()
trainer.train("model/uspto_random_mlm_#6/checkpoint-3600000")

Loading model from model/uspto_random_mlm_#6/checkpoint-3600000.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 449755
  Num Epochs = 200
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5622000
  Number of trainable parameters = 83504416
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 128
  Continuing training from global step 3600000
  Will skip the first 128 epochs then the first 1920 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch comm

  0%|          | 0/1920 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
3605000,0.053100,0.051214
3610000,0.052400,0.050244
3615000,0.052600,0.050595
3620000,0.051200,0.050926
3625000,0.052800,0.050425
3630000,0.051400,0.051423
3635000,0.051700,0.049991
3640000,0.053400,0.049604
3645000,0.055700,0.050464
3650000,0.054200,0.050287


***** Running Evaluation *****
  Num examples = 24986
  Batch size = 8
***** Running Evaluation *****
  Num examples = 24986
  Batch size = 8
***** Running Evaluation *****
  Num examples = 24986
  Batch size = 8
***** Running Evaluation *****
  Num examples = 24986
  Batch size = 8
Saving model checkpoint to model/uspto_random_mlm_#6/checkpoint-3620000
Configuration saved in model/uspto_random_mlm_#6/checkpoint-3620000/config.json
Model weights saved in model/uspto_random_mlm_#6/checkpoint-3620000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 24986
  Batch size = 8
***** Running Evaluation *****
  Num examples = 24986
  Batch size = 8
***** Running Evaluation *****
  Num examples = 24986
  Batch size = 8
***** Running Evaluation *****
  Num examples = 24986
  Batch size = 8
Saving model checkpoint to model/uspto_random_mlm_#6/checkpoint-3640000
Configuration saved in model/uspto_random_mlm_#6/checkpoint-3640000/config.json
Model weights saved in model/uspto_random_

In [ ]:
# trainer.evaluate()
trainer.save_model("model/{}_{}_mlm_#{}".format(learn_from, mask_method, trial_num))
wandb.finish()
torch.cuda.empty_cache()

# From saved model


In [ ]:
from transformers import pipeline
fill_mask = pipeline(
    "fill-mask",
    model="model/{}_{}_mlm_#{}".format(learn_from, mask_method, trial_num),
    # model="model/complete_random_mlm_#0/checkpoint-7660000",
    # tokenizer="tokenizer/{}_tokenizer".format('complete'),num_workers=12,device=0,batch_size=64
    tokenizer="tokenizer/{}_tokenizer".format(learn_from),num_workers=8,device=0,batch_size=64
)

if mask_method == 'random':
    # get data
    test_size = 0.1
    if learn_from == 'complete':
        test_size = 0.01
    dataset_trn, dataset_evl, dataset_tst = get_dataset(
        'default', masked_data_file, data_file, tokenizer, test_size=test_size)

# Training set results

In [ ]:
inputs_trn = [i['text'] for i in dataset_trn][1000::]
label_trn = [i['label'] for i in dataset_trn][1000::]

results= []
from tqdm import tqdm
i = 0
for item in tqdm(inputs_trn):
    results.append(fill_mask(item))
    i+=1
    if i>100:
        break

  0%|          | 100/574232 [00:02<3:34:05, 44.69it/s]


In [ ]:
correct = 0
total = 0
for i in range(1,100):
    if type(results[i][0])==dict:
        print('prediction = ',results[i][0]['token_str'])
        print('ground truth =',label_trn[i])
        print('\n')
        total+=1
        if results[i][0]['token_str']==label_trn[i]:
            correct+=1
print('the correct rate for one-token molecule is {}'.format(correct/total))

prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  O
ground truth = O


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  CO
ground truth = CO


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  O
ground truth = O


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  O
ground truth = O


prediction =  I
ground truth = I


prediction =  Cl
ground truth = Cl


prediction =  Cl
ground truth = Cl


prediction =  O
ground truth = O


prediction =  O
ground truth = O


prediction =  F
ground truth = F


prediction =  O
ground truth = O


prediction =  Cl
ground truth = Cl


prediction =  Br


In [ ]:
correct = 0
total = 0
for i in range(1,100):
    if type(results[i][0])!=dict:

        predict = [item for item in results[i]]

        if len(predict)<=10:
          predict = ''.join([item[0]['token_str'] for item in predict])
          print('prediction = ',predict)
          print('ground truth =',label_trn[i])
          print('\n')
          total+=1
          if predict==label_trn[i]:
              correct+=1
print('the correct rate for middle length token molecule is {}'.format(correct/total))

prediction =  N#N
ground truth = N#N


prediction =  O=O
ground truth = O=O


prediction =  O=O
ground truth = O=O


prediction =  O=O
ground truth = O=O


prediction =  CCCC(=O)O
ground truth = CCCC(=O)O


prediction =  O=O
ground truth = O=O


prediction =  O=O
ground truth = O=O


prediction =  CCccC
ground truth = CCCC(=O)O


prediction =  O=O
ground truth = O=O


prediction =  O=O
ground truth = O=O


prediction =  I.I
ground truth = [I-]


prediction =  O-2+2
ground truth = [O-2]


prediction =  O=O
ground truth = O=O


prediction =  O=O
ground truth = O=O


prediction =  O=O
ground truth = O=O


prediction =  C=CC(=O)O
ground truth = C=CC(=O)O


the correct rate for middle length token molecule is 0.8125


In [ ]:
correct = 0
total = 0
for i in range(1,100):
    if type(results[i][0])!=dict:

        predict = [item for item in results[i]]

        if len(predict)>10:
          predict = ''.join([item[0]['token_str'] for item in predict])
          print('prediction = ',predict)
          print('ground truth =',label_trn[i])
          print('\n')
          total+=1
          if predict==label_trn[i]:
              correct+=1
print('the correct rate for long length token molecule is {}'.format(correct/total))

prediction =  FC(F)(F)c()(cccc(12)2)2)c1
ground truth = OC(CNc1cccc(Oc2ccccc2)c1)C(F)(F)F


prediction =  FC(F)(F)c1cccc((((((((FFFFFF(F(FFFFFF))2)c1
ground truth = FC(F)(F)c1cccc(CN2CCCN(c3ccc4nnc(C(F)(F)F)n4n3)CC2)c1


prediction =  CC(C)c1cc(-c2ccccccccccc2)c2ncccc2c1
ground truth = CC(C)c1cc(-c2ccc(F)c(C=O)c2)c2ncccc2c1


prediction =  COC(O)c1ccc(O((((N(C))C)cc1
ground truth = COC(=O)c1ccc(NS(=O)(=O)N(C)C)cc1


prediction =  COc1ccc2c3c1O[[[[[C[CCHCHCCCCCCCHCCCCCCCCCCCCCCCCCCC
ground truth = COc1ccc2c3c1O[C@H]1[C@@H](O)C=C[C@H]4[C@@H](C2)N(C)CC[C@]314


prediction =  O=c1[nH]ccc)c222ccccccccccc22222222(F)F)cccc1F
ground truth = O=c1[nH]cc(Cl)c2c1c(-c1ccc(N3CCOCC3)cc1)nn2-c1c(F)cccc1F


prediction =  CCOC(=O)c1nc(N)cn1CC1CC1
ground truth = CCOC(=O)c1nc(N)cn1CC1CC1


prediction =  CCOC(=O)CCN(Cc1ccccc1)Cc1ccccc1
ground truth = CCOC(=O)CCN(Cc1ccccc1)Cc1ccccc1


prediction =  COc1ccc2c3c(c(c(c()))))CC))))))((()))O)c1-3
ground truth = COc1ccc2c3c(c(NCCN(C)C)nc2c1)C(=O)c1ccnc(NCCN(C)C)c

# Test set results

In [ ]:
inputs_tst = [i['text'] for i in dataset_tst][300::]
label_tst = [i['label'] for i in dataset_tst][300::]

results= []
from tqdm import tqdm
i = 0
for item in tqdm(inputs_tst):
    results.append(fill_mask(item))
    i+=1
    if i>100:
        break

  0%|          | 100/79880 [00:03<41:07, 32.33it/s]


In [ ]:
correct = 0
total = 0
for i in range(1,100):
    if type(results[i][0])==dict:
        print('prediction = ',results[i][0]['token_str'])
        print('ground truth =',label_tst[i])
        print('\n')
        total+=1
        if results[i][0]['token_str']==label_tst[i]:
            correct+=1
print('the correct rate for one-token molecule is {}'.format(correct/total))

prediction =  2
ground truth = CCO


prediction =  O
ground truth = Cl


prediction =  O
ground truth = O


prediction =  O
ground truth = O


prediction =  Cl
ground truth = Cl


prediction =  Br
ground truth = Br


prediction =  F
ground truth = F


prediction =  Cl
ground truth = Cl


prediction =  CCO
ground truth = CCO


prediction =  O
ground truth = O


prediction =  CCO
ground truth = CCO


prediction =  Cl
ground truth = Cl


prediction =  Br
ground truth = Br


prediction =  O
ground truth = O


prediction =  I
ground truth = I


prediction =  Cl
ground truth = Cl


prediction =  O
ground truth = O


prediction =  O
ground truth = O


prediction =  O
ground truth = O


prediction =  O
ground truth = O


prediction =  Cl
ground truth = Cl


prediction =  Br
ground truth = Br


prediction =  Cl
ground truth = Cl


prediction =  O
ground truth = O


prediction =  Cl
ground truth = Cl


prediction =  O
ground truth = O


prediction =  Cl
ground truth = Cl


the correct rate for o

In [ ]:
correct = 0
total = 0
for i in range(1,100):
    if type(results[i][0])!=dict:

        predict = [item for item in results[i]]

        if len(predict)<=10:
            predict = ''.join([item[0]['token_str'] for item in predict])
            print('prediction = ',predict)
            print('ground truth =',label_tst[i])
            print('\n')
            total+=1
            if predict==label_tst[i]:
                correct+=1
print('the correct rate for middle length token molecule is {}'.format(correct/total))

prediction =  CC(=O)O
ground truth = [H][H]


prediction =  O.[Mn].O
ground truth = [SbH3]


prediction =  Fe].[Fe
ground truth = [Y]


prediction =  Cc1ccc(N)cc1
ground truth = CNc1ccc(C)cc1


prediction =  CC(=O)O
ground truth = CC(=O)O


prediction =  CCCCC1([1-].O
ground truth = CCCCC(=O)[O-]


prediction =  NC(=NO)COc1ccccc1
ground truth = NC(COc1ccccc1)=NO


prediction =  B>>Co
ground truth = [Co]


prediction =  CC(=O)O
ground truth = CC(=O)O


prediction =  CC(=O)]
ground truth = [H][H]


prediction =  Cl[H[HH])([2)Cl
ground truth = [H]C([H])(Cl)Cl


prediction =  CCC(=O)O
ground truth = CCC(=O)O


prediction =  CCCCCCCCCCCCCCCCCCCCCC[Sn](Cl)(Cl)Cl
ground truth = CCCCCCCCCCCCCCCCCCCCCC[Sn](Cl)(Cl)Cl


prediction =  Fe3].[].3
ground truth = [BiH3]


prediction =  CCC(=O)O
ground truth = CCC(=O)O


prediction =  Cl.Cl
ground truth = [Cl-]


prediction =  O=O
ground truth = O=O


prediction =  C=C
ground truth = C=C


the correct rate for middle length token molecule is 0.38888888

In [ ]:
correct = 0
total = 0
for i in range(1,100):
    if type(results[i][0])!=dict:

        predict = [item for item in results[i]]

        if len(predict)>10:
            predict = ''.join([item[0]['token_str'] for item in predict])
            print('prediction = ',predict)
            print('ground truth =',label_tst[i])
            print('\n')
            total+=1
            if predict==label_tst[i]:
                correct+=1
print('the correct rate for long length molecule is {}'.format(correct/total))

prediction =  CCC/C=C/Cn1cccc1C=O
ground truth = CCC/C=C/Cn1cccc1C=O


prediction =  CC=O)COP(=O)(O)O
ground truth = CC(=O)OP(=O)(O)O


prediction =  O=C1C[C@@H2)22CCCCCCCCCC)))))))))))))))))))))))))))))))))))))))))))))))))))c1ccccc1
ground truth = O=C1C[C@@H](c2ccccc2)[C@]23C(=O)CCC[C@@]2(O)[C@H]([N+](=O)[O-])[C@@H](c2ccc(Br)cc2)[C@@]2(C(=O)N(Cc4ccccc4)c4ccccc42)[C@H]13


prediction =  CCCCC#Cc1cc(F)O)O1O11111(1)(C)cc1
ground truth = CCCCC#Cc1cc(F)ccc1C=NNS(=O)(=O)c1ccc(C)cc1


prediction =  CC(=O)n1c2c(C)C)))))))))))))))))))))C(n2C
ground truth = C=C(OC(C)=O)n1c(N(C)C)nc2c1c(=O)n(C)c(=O)n2C


prediction =  CCOC(=O)C1nc2ccccc2O3OOO)CCCC)O)c1=O.O
ground truth = CCOC(=O)[C@]1([C@H](C=O)Cc2ccccc2)Nc2ccccc2OC1=O


prediction =  CSc1nc(SC)c2c(Br(C(((((C)C)n1
ground truth = CSc1nc(SC)c2c(Br)nn(CCCCOC(C)=O)c2n1


prediction =  O=C(O[C]H]HOOOOOOHOOOOO))))))))))O)c1ccccc1
ground truth = O=C(O[C@@H](C(=O)O)[C@@H](OC(=O)c1ccccc1)C(=O)O)c1ccccc1


prediction =  CCC1cccc2c1C1CCCCCCCC11C11111C1111C